In [1]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'
spark_version = 'spark-3.4.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:2 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:5 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [108 kB]
Get:7 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [734 kB]
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:9 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [731 kB]
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 1,802 kB in 3s (571 kB/s)
Reading package lists... Done


In [2]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [6]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"

spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True)
df.show()


+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [7]:
# 2. Create a temporary view of the DataFrame.
df.createOrReplaceTempView('homes')


In [8]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?

query = """
    SELECT
        year(date) AS year_sold,
        avg(price) AS avg_price
    FROM
        homes
    WHERE
        bedrooms = 4
    GROUP BY
        year_sold
    ORDER BY
        year_sold
"""

result = spark.sql(query)

result = result.withColumn("avg_price", result["avg_price"].cast("decimal(10,2)"))
result.show()


+---------+---------+
|year_sold|avg_price|
+---------+---------+
|     2019|300263.70|
|     2020|298353.78|
|     2021|301819.44|
|     2022|296363.88|
+---------+---------+



In [9]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?

query = """
    SELECT
        year(date_built) AS year_built,
        avg(price) AS avg_price
    FROM
        homes
    WHERE
        bedrooms = 3 AND bathrooms = 3
    GROUP BY
        year_built
    ORDER BY
        year_built
"""

result = spark.sql(query)

result = result.withColumn("avg_price", result["avg_price"].cast("decimal(10,2)"))
result.show()


+----------+---------+
|year_built|avg_price|
+----------+---------+
|      2010|292859.62|
|      2011|291117.47|
|      2012|293683.19|
|      2013|295962.27|
|      2014|290852.27|
|      2015|288770.30|
|      2016|290555.07|
|      2017|292676.79|
+----------+---------+



In [10]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?

query = """
    SELECT
        year(date_built) AS year_built,
        avg(price) AS avg_price
    FROM
        homes
    WHERE
        bedrooms = 3
        AND bathrooms = 3
        AND floors = 2
        AND sqft_living >= 2000
    GROUP BY
        year_built
    ORDER BY
        year_built
"""


result = spark.sql(query)

result = result.withColumn("avg_price", result["avg_price"].cast("decimal(10,2)"))
result.show()


+----------+---------+
|year_built|avg_price|
+----------+---------+
|      2010|285010.22|
|      2011|276553.81|
|      2012|307539.97|
|      2013|303676.79|
|      2014|298264.72|
|      2015|297609.97|
|      2016|293965.10|
|      2017|280317.58|
+----------+---------+



In [13]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()

query = """
    SELECT
        avg(view) AS avg_view_rating
    FROM
        homes
    WHERE
        price >= 350000

"""

result = spark.sql(query)

result = result.withColumn("avg_view_rating", result["avg_view_rating"].cast("decimal(10,2)"))
result.show()


print("--- %s seconds ---" % (time.time() - start_time))



+---------------+
|avg_view_rating|
+---------------+
|          32.26|
+---------------+

--- 0.8158388137817383 seconds ---


In [14]:
# 7. Cache the the temporary table home_sales.

spark.catalog.cacheTable("homes")


In [16]:
# 8. Check if the table is cached.
spark.catalog.isCached('homes')

True

In [18]:
# 9. Using the cached data, run the query that filters out the view ratings with average price
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()

query = """
    SELECT
        avg(view) AS avg_view_rating
    FROM
        homes
    WHERE
        price >= 350000

"""

result = spark.sql(query)

result = result.withColumn("avg_view_rating", result["avg_view_rating"].cast("decimal(10,2)"))
result.show()

print("--- %s seconds ---" % (time.time() - start_time))


+---------------+
|avg_view_rating|
+---------------+
|          32.26|
+---------------+

--- 0.30086565017700195 seconds ---


In [21]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
from pyspark.sql.functions import col

df = spark.table("Homes")
df_partitioned = df.repartition(col("date_built"))


In [23]:
# 11. Read the parquet formatted data.

df_partitioned.show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e81aacfe-17fe-46b...|2020-06-16|      2016|181925|       3|        3|       2137|   11709|     2|         0|  22|
|dd61eb34-6589-4c0...|2021-07-25|      2016|210247|       3|        2|       1672|   11986|     2|         0|  28|
|f233cb41-6f33-4b0...|2021-07-18|      2016|437375|       4|        3|       1704|   11721|     2|         0|  34|
|4566cd2a-ac6e-435...|2019-07-15|      2016|177541|       3|        3|       213

In [24]:
# 12. Create a temporary table for the parquet data.

df.createOrReplaceTempView("Homes_parquet")


In [25]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000
# with the parquet DataFrame. Round your average to two decimal places.
# Determine the runtime and compare it to the cached version.

start_time = time.time()

query = """
    SELECT
        avg(view) AS avg_view_rating
    FROM
        Homes_parquet
    WHERE
        price >= 350000

"""

result = spark.sql(query)

result = result.withColumn("avg_view_rating", result["avg_view_rating"].cast("decimal(10,2)"))
result.show()


print("--- %s seconds ---" % (time.time() - start_time))

+---------------+
|avg_view_rating|
+---------------+
|          32.26|
+---------------+

--- 0.44986891746520996 seconds ---


In [26]:
# 14. Uncache the home_sales temporary table.

spark.catalog.uncacheTable("Homes")

In [27]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached('homes')


False